In [2]:
import platform
import sys
print("This jupyter notebook is running on Python " + platform.python_version())
# It's good practice to assert packages requirements at the beginning of a script:
assert sys.version_info >= (3, 6) # Tested with Python==3.7.5
!pip install twitter PrettyTable matplotlib

This jupyter notebook is running on Python 3.7.4
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import twitter # Tell Python to use the twitter package
CONSUMER_KEY = 'mTLDFe8vELJ3dJ0kjfdzUMJri'
CONSUMER_SECRET = '3rZf0h5kknvcPt7PhsYfrPVaj68lIY8Ko34Xhf74veB1chdNpc'
# to get the oauth credential you need to click on the 'Generate access token' button:
OAUTH_TOKEN = '1225124335452590080-ldbeNYWS2nZ8zSNeEllQOCkVYQpFyJ' 
OAUTH_TOKEN_SECRET = 'wygh7OKoeP0bP75zNFOB6IEQUJbaVOSYQcDTxULVKyQmR'
auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)
print(twitter_api) 

In [13]:
import pandas as pd
q = '#coronavirus' # XXX: Set this variable to a trending topic, or anything else you like. 
count = 100 # number of results to retrieve

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets for more info

search_results = twitter_api.search.tweets(q=q, count=count) # search for your query 'q' 100 times
statuses = search_results['statuses'] # extract the tweets found

# The following code allows you to print in a nice format the contents of search_results
import json
print(type(statuses))
print(json.dumps(statuses[0], indent=1))
df = pd.read_json(json.dumps(statuses[0], indent=1)).dropna(axis=1, thresh = count/2)
print(df)
print(df.describe())

<class 'list'>
{
 "created_at": "Thu Feb 27 11:16:58 +0000 2020",
 "id": 1232987986712768512,
 "id_str": "1232987986712768512",
 "text": "RT @aucarr: Il y a vraiment des gens qui font une fellation pour dire bonjour ? #coronavirus",
 "truncated": false,
 "entities": {
  "hashtags": [
   {
    "text": "coronavirus",
    "indices": [
     80,
     92
    ]
   }
  ],
  "symbols": [],
  "user_mentions": [
   {
    "screen_name": "aucarr",
    "name": "Aucarr",
    "id": 1096767212,
    "id_str": "1096767212",
    "indices": [
     3,
     10
    ]
   }
  ],
  "urls": []
 },
 "metadata": {
  "iso_language_code": "fr",
  "result_type": "recent"
 },
 "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
 "in_reply_to_status_id": null,
 "in_reply_to_status_id_str": null,
 "in_reply_to_user_id": null,
 "in_reply_to_user_id_str": null,
 "in_reply_to_screen_name": null,
 "user": {
  "id": 1213698534513369089,
  "id_str": "1213698534513369089",
  "na

In [14]:
status_texts = [ status['user'] for status in statuses ]
# the escape character "\" allows for the list comprehension to continue
# on a new line. While not strictly necessary, it makes code more readable
# for your fellow programmers.
userdf = pd.read_json(json.dumps(status_texts, indent=1))['location']

df = pd.concat([df, userdf], axis=0)
print(df)
print(df.describe())
print(userdf)


                                            0                created_at  \
hashtags                                  NaN 2020-02-27 11:16:58+00:00   
symbols                                   NaN 2020-02-27 11:16:58+00:00   
user_mentions                             NaN 2020-02-27 11:16:58+00:00   
urls                                      NaN 2020-02-27 11:16:58+00:00   
iso_language_code                         NaN 2020-02-27 11:16:58+00:00   
...                                       ...                       ...   
95                                        Ksa                       NaT   
96                                        usa                       NaT   
97                                                                  NaT   
98                                                                  NaT   
99                 I follow back 🇺🇸 🚫NO Dates                       NaT   

                   favorite_count favorited            id        id_str  \
hashtags                

In [15]:
print(df.columns)

Index([                     0,           'created_at',       'favorite_count',
                  'favorited',                   'id',               'id_str',
            'is_quote_status',                 'lang',     'quoted_status_id',
       'quoted_status_id_str',        'retweet_count',            'retweeted',
                     'source',                 'text',            'truncated'],
      dtype='object')


In [16]:
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 931kB 6.3MB/s eta 0:00:01     |███████████████████▍            | 563kB 6.3MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 2.0MB/s eta 0:00:01     |██████████████████████▍         | 1.1MB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 13.9MB 5.3MB/s eta 0:00:011   |██████▏                         | 2.7MB 7.4MB/s eta 0:00:02     |█████████████                   | 5.7MB 3.7MB/s eta 0:00:03     |████████████████████████        | 10.4MB 10.1MB/s eta 0:00:01     |████████████████████████████████| 13.9MB 5.3MB/s 
     |████████████████████████████████| 11.0MB 10.9MB/s eta 0:00:01   |███████████████▋                | 5.3MB 4.6MB/s eta 0:00:02
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 112kB 550kB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [ ]:
map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1